### importing the necessary dependencies and activating them

In [ ]:
import pathlib
import pandas as pd
import panel as pn
import xarray as xr
import holoviews as hv

pn.extension('tabulator')
pn.extension()
import colorcet as cc
import hvplot.xarray # noqa
import hvplot.pandas # noqa

### loading the dataset **UN SDG maternal health risk data set**
> source; [kaggle](kaggle.com)

In [ ]:
df = pd.read_csv("maternal health risk data set.csv")
df.head()

#### checking if there are NaN values or ommitted values in the data set

In [ ]:
df.isna().sum()

In [ ]:
df['Age'].unique()

#### taking the overall statistics of data.

In [ ]:
df.describe()

#### calculating the correlation between age of the pregnant women and their blood sugar levels.

In [ ]:
cor_age_bs = df['Age'].corr(df['BS'])
cor_age_bs

#### calculating the correlation between age of the pregnant women and their systolic BP

In [ ]:
cor_age_syst = df['Age'].corr(df['SystolicBP'])
cor_age_syst

#### calculating the correlation between the systolic BP of the pregnant women and their heart rate

In [ ]:
cor_syst_hrtrt = df['SystolicBP'].corr(df['HeartRate'])
cor_syst_hrtrt

#### regressing age of the women against their blood sugar and heartrate
* using statsmodels
* linear regression
* OLS model

In [ ]:
import statsmodels.api as sm

In [ ]:
x = df[['BS', 'HeartRate']]
y = df['Age']
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

### writing a function to select a risk level in order to bind it with a widget

In [ ]:
def select_risk(risk_level):
    return df[df['RiskLevel'] == risk_level]
    

#### creating a widget to toggle the risks

In [ ]:
risk_widget = pn.widgets.Select(name='risk selector', options=list(df['RiskLevel'].unique()))

### binding the widget with the function

In [ ]:
risk_df = pn.bind(select_risk, risk_widget)
risk_df

#### calculating the percentages of the risks factors.

In [ ]:
low = select_risk('low risk')
total_rows = len(low)
per1 = (total_rows/1014)* 100

high = select_risk('high risk')
total_rows = len(high)
per2 = (total_rows/1014)* 100

mid = select_risk('mid risk')
total_rows = len(mid)
per3 = (total_rows/1014)* 100

#### displaying the data with the widget 

In [ ]:
pn.Column(risk_widget, risk_df)

### writing a function to make a scatter plot to react with the widget

In [ ]:
def plot_risk(risk_level):
    risk = select_risk(risk_level)
    return risk.hvplot.scatter(x='Age', y=['SystolicBP', 'DiastolicBP', 'BS', 'HeartRate', 'BodyTemp'], 
                               title='The BP levels, BS, Heart Rate, & Body temp of the pregnant women')

### displaying the scatter plot using pn.Row

In [ ]:
risk_scatter = pn.bind(plot_risk, risk_widget)
pn.Row(risk_scatter, risk_widget)

### a function to plot a line graph of the systolic and diastolic BP of the women

In [ ]:
def plot_sys_dia(risk_level):
    risk = select_risk(risk_level)
    return risk.hvplot.line(y=['SystolicBP', 'DiastolicBP'], title='The systolic and diastolic distribution of the pregnant women')

### displaying the plot with the widget

In [ ]:
bp_plot = pn.bind(plot_sys_dia, risk_widget)
pn.Row(bp_plot, risk_widget)

### A hexbin plot grouped by the risk level

In [ ]:
df.hvplot.hexbin(x = 'Age',  y='BS', groupby='RiskLevel')

### Creating a table of the data for use in the dashboard

In [ ]:
df_table = pn.widgets.Tabulator(df, buttons={'Print': "<i class='fa fa-print'></i>"},)
df_table

## Creating an indicator of the risk levels

In [ ]:
high_risk = pn.indicators.Number(
    name = 'High Risk', value=26.8, format='{value}%',
    colors= [(30, 'red'), (40, 'gold'), (50, 'green')]
)
mid_risk = pn.indicators.Number(
    name = 'Mid Risk', value=33.1, format='{value}%',
    colors= [(30, 'red'), (40, 'gold'), (50, 'green')]
)
low_risk = pn.indicators.Number(
    name = 'Low Risk', value=40.1, format='{value}%',
    colors= [(30, 'red'), (40, 'gold'), (50, 'green')]
)

indicator = pn.Column(high_risk, mid_risk, low_risk)

# creating and deploying a fine dashboard

In [ ]:
template = pn.template.FastListTemplate(
    title="UN SDG Maternal Health risk in Bangladesh",
    sidebar=[risk_widget, indicator],
)
# Append a layout to the main area, to demonstrate the list-like API
template.main.append(
    pn.Column(
        pn.bind(plot_risk, risk_widget),
        pn.bind(plot_sys_dia, risk_widget),df.hvplot.hexbin(x = 'Age',  y='BS', groupby='RiskLevel'), df_table,
    )
)
template.show();